<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

# Klassifikation mit Logistischer Regression

Bisher haben wir gesehen, wie man mit der linearen Regression Schätzfunktionen für lineare Modelle aufstellen und mithilfe des Gradientenverfahrens trainieren kann.
In diesem Arbeitsblatt wollen wir nun das Problem der Klassifikation betrachten.
Beispiele für Klassifikationsaufgaben sind:
- Werbung/Marketing: Wird ein Kunde ein bestimmtes Produkt kaufen?
- Qualitätssicherung: Ist ein bestimmtes Produkt ok oder defekt?
- Objekterkennung: Ist ein bestimmtes Objekt auf einem Bild zu sehen?
- Betrugserkennung: Liegt bei einer bestimmten Transaktion ein Betrugsfall vor?
- Finanzanalysen: Zahlt ein Kreditnehmer einen Kredit vollständig zurück?
- Medizin: Hat ein Patient eine bestimmte Krankheit?
- ...

Anders als bei der Regression, wo wir für einen neuen Datenpunkt einen möglichst genauen Schätzwert für eine abhängige Variable berechnet haben, geht es bei der Klassifikation darum, vorherzusagen, zu welcher Klasse der Datenpunkt gehört.
Statt einer kontinuierlichen Zielgröße wird also bei der Klassifikation eine diskrete abhängige Variable vorhergesagt.
Im einfachsten Fall ist die zu erklärende Variable binär, die Schätzungen haben also nur die Werte *ja* oder *nein*, bzw. `1` oder `0`.
Bei der *Multiklassen-Klassifikation* kann die Zielvariable mehr als 2 diskrete Werte annehmen, die jeweils eine bestimmte Klasse kodieren.
Beispielsweise kann ein Bilderkennungssystem vorhersagen, welches von 100 bekannten Objekten sich am wahrscheinlichsten auf einem Bild befindet.
Werden für die Schätzung der Zielgröße, wie es üblicherweise der Fall ist, mehrere unabhängige Variable herangezogen, spricht auch von einer multivariaten Klassifikation.

Grundsätzlich kann man ein Klassifikationsproblem auch mithilfe der *linearen Regression* angehen.
Z.B. indem man eine Schätzfunktion $f$ für eine binäre Zielvariable erstellt wobei man für die Vorhersage den Wert $f(x)$ als Wahrscheinlichkeit für die Zugehörigkeit  des Datenpunkt $x$ zur Klasse `1` interpretiert. D.h., die Werte $f(x)<0.5$ werden als `0`, die Werte $f(x)\ge0.5$ als `1` interpretiert. 
Allerdings ergeben sich dadurch einige Probleme, die die lineare Regression für Klassifikationsaufgaben nicht sehr praktikabel machen.
U.a. liefert die Schätzfunktion $f$ auch Werte kleiner `0` und größer `1`, was bei der Interpretation als Wahrscheinlichkeit widersinnig ist.

Die logistische Regression löst das Problem der Schätzfunktion indem Sie das Ergebnis der linearen Funktion durch eine geeignete Transformation auf den Wertebereich `0` bis `1` abgebildet.
Diese Transformation wird bei logistische Regression der logistischen Funktion (auch *Sigmoidfunktion* oder *S-Funktion*) durchgeführt.

### Logistische Regression

Um die Methode der logistischen Regression genauer zu erklären, generieren wir uns einenen synthetischen Datensatz mit nur zwei Merkmalen.
Diese vereinfachte Problemstellung erlaubt es uns, Daten und Funktionen im zweidimensionalen Koordinatensystem zu plotten und so besser zu visualisieren.

Unser frei erstellter Datensatz soll einen Zusammenhang zwischen der Zeit, die ein Student für eine Prüfung lernt und dem Prüfungsresultat beschreiben.
Wir verwenden zur Erzeugung der Datenpunkte die Sigmoidfunktion `sigma(x)`, die auch später die Grundlage des logistischen Regressionsmodells ist.

Die Sigmoidfunktion kann in Python wiefolgt implementiert werden.
(Der Zusatz @np.vectorize bewirkt, dass die Funktion bei Eingabe eines NumPy Arrays für alle Elemente des Arrays in vektorisierter Form ausgeführt wird.)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

@np.vectorize
def sigma(x):
    return 1.0 / (1.0+np.exp(-x))

xx = np.arange(-10, 10, .1)

fig, ax =plt.subplots(1, 1, figsize=(9,5))

plt.plot(xx, sigma(xx), linewidth=4)
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['left'].set_position(('data',0))
ax.spines['bottom'].set_position(('data',0))
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontsize(12)
    label.set_bbox(dict(facecolor='white', edgecolor='None', alpha=0.65 ))


plt.savefig("LogistischeRegression01.png",transparent=True, dpi=300)

Um die Verteilung der Datenpunkte etwas zu variieren, addieren wir zum Ergebnis der Sigmoidfunktion noch eine normalverteilte Störgröße `sigma_z_noise`.

Der zugehörige Graph zeigt die Resultate (`0=`*nicht bestanden* und `1=`*bestanden*) auf der y-Achse und die Anzahl der Lernstunden auf der y-Achse.
Wie man sieht, ist es nicht möglich, die eine Trennlinie, nur in Anhängigkeit von dem Merkmal *Lernstunden* zu ziehen, die die Datenpunkte in die Klassen *nicht bestanden* und *bestanden* aufteilt. Ein Wert von ca. 30 scheint eine gute Wahl zu sein, aber auch bei dieser Aufteilung gibt es einige Datenpunkte, die in die "falsche" Kategorie fallen.

In [ ]:
import matplotlib.pyplot as plt

np.random.seed(12)
#np.random.seed(12)

@np.vectorize
def bestehen(x):
    z = x/10-3
    sigma_z = 1.0 / (1.0+np.exp(-z))
    sigma_z_noise = sigma_z + np.random.normal(0,.4)
    #sigma_z_noise = sigma_z + np.random.normal(.1,.2)
    if sigma_z_noise < 0.3: return 0
    else: return 1

X = np.random.uniform(0,80,40)
y = bestehen(X)
plt.scatter(X[y==0], y[y==0], marker='o')
plt.scatter(X[y==1], y[y==1], marker='d')
plt.xlabel("Lernstunden", fontsize=14)
plt.text(3, .9, "bestanden")
plt.text(50, .05, "nicht bestanden")
plt.axvline(x=30, c='grey', ls=':', label="Entscheidungsgrenze")
plt.savefig("LogistischeRegression03.png",transparent=True, dpi=300)
plt.show()


Da wir noch nicht wissen, wie wir eine Funktion herleiten können, die auf Grundlage der Lernstunden die Wahrscheinlichkeitsverteilung für das Bestehen der Klausur voraussagt, überlegen wir, wie so eine Funktion aussehen könnte.
Im Diagramm unten abgebildet, ist der Graph der Funktion `prob`, die eine Schätzung der Wahrscheinlichkeitsverteilung für das Bestehen der Klausur unter Angabe der Lernstunden darstellt.

Diese partiell lineare Funktion ist ein mögliches Modell für die Wahrscheinlichkeiten eines Erfolgs.
Wenn man nun die Entscheidungsgrenze bei `prob(x)=0.5` anlegt sieht man, dass die Funktion suboptimal ist.

In [ ]:
epsilon = 1e-3

miny = X[y==1].min()
maxy = X[y==0].max()


@np.vectorize
def prob(x,a,b):
    a = a-1
    b = b+1
    if x<a: return 0.0+epsilon
    elif x>b: return 1.0-epsilon
    else:
        res = 1/(b-a) * x - (a/(b-a))
        assert res<1, "Für x=%f ergibt die W'keit 1" % x
        return res
    
xx = np.linspace(-10,100,100)
plt.scatter(X, y)
plt.xlabel("Lernstunden", fontsize=14)
plt.plot(xx, prob(xx,miny,maxy), c='r')
plt.axvline(x=miny+(maxy-miny)*.5, c='grey', ls=':')
#plt.axhline(y=.5, xmin=.0, xmax=.5, c='black', ls='--')

plt.savefig("LogistischeRegression04.png",transparent=True, dpi=300)
plt.show()
miny, maxy

Wir verwenden nun unsere geschätzte Wahrscheinlichkeitsverteilung, um die *Chancen* (auf einen Erfolg) zu berechnen.
In der Statistik beschreibt die Chance (engl *odds*) den Quotienten aus der Wahrscheinlichkeit $p$ eines Ereignisses und seiner Gegenwahrscheinlichkeit: $\frac{p}{1-p}$.

Beim Münzwurf beträgt die Chance z.B.  1:1 (Wahrscheilichkeit $\frac{0,5}{0,5}$ oder "ein guter Fall, ein schlechter Fall"), beim Würfeln einer sechs 1:5.

Bei einer Wahrscheinlichkeit von 0 ist die Chance ebenfalls 0 ($\frac{0}{1}$). Je weiter sich die Wahrscheinlichkeit für einen Erfolg der 1 nähert, desto größer wird die Chance: $\lim\limits_{p \rightarrow 1}{\frac{p}{1-p}}=\infty$

In [ ]:
@np.vectorize
def chance(x):
    assert x<1, "Für x=%f sind die Chancen nicht definiert" % x
    return x/(1-x)

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

xx = np.linspace(miny+epsilon,maxy-epsilon,100)
ax1.scatter(X, y)

xxx = prob(xx,miny,maxy)

ax1.set_xlabel("Lernstunden", fontsize=14)
ax2.set_xlabel(r"$p$", fontsize=14)
ax2.plot(xxx, chance(xxx), c='r', label="Chance")
plt.legend(loc='upper left')
plt.savefig("LogistischeRegression_Chance.png",transparent=True, dpi=300)
ax1.axis([0,80,-0.1,10])

Im obigen Graph sind auf der x-Achse die Lernstunden für die Datenpunkte sowie die Bestehens-Wahrscheinlichkeit $p$ für die Funktion *Chance* aufgetragen.
Unabhängig von dem Anwendungsfall besitzt *Chance* für jede lineare Wahrscheinlichkeitsfunktion $P(X=x_i)$ die gleiche Form.


Nun transformieren wir die Chance-Funktion mit dem (natürlichen) Logarithmus aus dem Wertebereich $[0,\infty[$ in den Bereich $]-\infty,\infty[$.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

ax1.scatter(X, y)
ax1.set_xlabel("Lernstunden", fontsize=14)
ax2.set_xlabel(r"$p$", fontsize=14)
xxx = prob(xx,miny,maxy)
ax2.plot(xxx, chance(xxx), c='r', label="Chance")
ax2.plot(xxx, np.log(chance(xxx)), c='g', label="logit")
plt.legend(loc='upper left')
ax1.axis([0,80,-5,5])
plt.savefig("LogistischeRegression06.png",transparent=True, dpi=300)
plt.show()

Die resultierende **logit** Funktion läuft "in der Mitte", also ca. dem Bereich $[0.2,0.8]$ annähernd linear.
Daher können wir die Kurve durch ein lineares Modell approximieren.

In [ ]:
xxx = prob(X,miny,maxy)
xxx

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()

xxx = prob(X,miny,maxy)
XX = X.reshape(-1, 1)
yy = np.log(chance(xxx)).reshape(-1, 1)
    
linreg.fit(XX,yy)
t0 = linreg.intercept_
t1 = linreg.coef_
t0, t1

Im unten angegebenen Graph sind *Chance* und *logit* Funktionen der Wahrscheinlichkeit.
Die Funktion *model* hingegen, hängt von den Lernstunden ab.
Man sieht, dass sich die Modellfunktion und die Logit-Funktion im Mittelteil recht gut überdecken.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_xlabel("Lernstunden", fontsize=14)
ax2.set_xlabel(r"$p$", fontsize=14)

ax1.scatter(X, y)
xxx = prob(xx,miny,maxy)
ax2.plot(xxx, chance(xxx), c='r', label="Chance")
ax2.plot(xxx, np.log(chance(xxx)), c='g', label="logit")
modely = list(map(lambda x: (t0+t1*x).item(0), X))
modelx, modely = zip(*sorted(zip(X, modely)))
ax1.plot(modelx, modely, c='orange', label="model", linewidth=3)
ax2.legend(loc='upper left')
ax1.legend(loc='lower right')
ax1.axis([0,80,-5,5])
plt.savefig("LogistischeRegression07.png",transparent=True, dpi=300)
plt.show()

Wir haben bisher mit der "selbst ausgedachten" Verteilungsfunktion für die Wahrscheinlichkeiten *prob* gearbeitet.
Durch Anwenden der Logit-Funktion auf *prob* haben wir eine Funktion erzeugt, die wir mit mit einem linearen Model $\Theta^Tx$ approximieren können.

Wir wollen nun betrachten, wie man die Wahrscheinlichkeitsfunktion allgemein bestimmen kann.
Wir wissen, dass sich die Logit-Funktion $logit(x)=ln(chance(x))$ äquivalent zu der lineatren Funktion $h_{\Theta}(x)=\Theta^Tx$ verhält. Wir können nun durch Einsetzen und Umformen die Verteilungsfunktion $p(x)$ herleiten:

$$ln(chance(x)) \thicksim \Theta^Tx$$

$$\Leftrightarrow chance(x) \thicksim e^{\Theta^Tx}$$

$$\Leftrightarrow \frac{p(x)}{1-p(x)} \thicksim e^{\Theta^Tx}$$

$$\Leftrightarrow \frac{p(x)}{1-p(x)} \cdot \frac{1/p(x)}{1/p(x)} \thicksim e^{\Theta^Tx}$$

$$\Leftrightarrow \frac{1}{p(x)^{-1}-1} \thicksim e^{\Theta^Tx}$$

$$\Leftrightarrow p(x)^{-1}-1 \thicksim e^{-\Theta^Tx}$$

$$\Leftrightarrow p(x)^{-1} \thicksim 1+e^{-\Theta^Tx}$$

$$\Leftrightarrow p(x) \thicksim \frac{1}{1+e^{-\Theta^Tx}}$$

Damit haben wir gezeigt, dass sich die Wahrscheinlichkeitsverteilung als Sigmoidfunktion in Abhängigkeit der Modellparameter $\Theta$ sowie den Werten der unabhängigen Variablen darstellen lässt.
Damit haben wir die **Modellfunktion** für unser Klassifikationsproblem erhalten:

$$
h_{\Theta}(x) = \frac{1}{1+e^{-\Theta^Tx}}
$$

Als nächsten Schtritt wollen wir nun betrachten, welche **Kostenfunktion** wir zur Bestimmung der Parameter $\Theta$ anwenden können.

Bei der linearen Regression haben wir als Kostenfunktion die Summe der Fehlerquadrate verwendet.
Schauen wir uns zunächst an, welche Kostenfunktion dieser Ansatz für die Sigmoidfunktion liefert.

Wir erweitern zuerst unsere Matrix `X` um eine Spalten mit Einsen für die Bestimmung des Bias-Parameters $\Theta_0$.

In [ ]:
vone = (np.ones(len(X))).reshape(-1,1)
XX = X.reshape(-1,1)
XX = np.concatenate((vone, XX), 1)
yy = y.reshape(-1,1)
# Und wieder zurück:
#X = X[:,1:]
#X = X.reshape(1,-1)[0]

Im folgenden Code-Abschnitt wird die Kostenfunktion $J_{\Theta}(x)$ geplottet.
Um die Funktion 2-dimensional darstellen zu können, setzen wir einen Parameter $\Theta_0$ fest.

In [ ]:
def h(X,theta):
    #print("X:", np.shape(X), "Theta:", np.shape(theta))
    r = 1.0 / (1.0+np.exp(-(X@theta)))
    #print("r:", np.shape(r))
    return r

def J(X,theta,y):
    #print("X:", np.shape(X), "Theta:", np.shape(theta), "y:", np.shape(y))
    j = (h(X,theta)-y).T@(h(X,theta)-y)
    #print("j:", np.shape(j))
    return j

yt1 = []
xt1 = []

for t in np.linspace(-2,2,1000):
    xt1.append(t)
    theata0 = np.array([33, t]).reshape((2,1))
    yt1.append(J(XX,theata0,yy).item(0))



plt.plot(xt1, yt1, label=r'$J_{\Theta}(x)$')
plt.legend(loc='upper right', prop={'size': 16})
plt.xlabel(r'$\Theta_1$', fontsize=16)
plt.text(0, 22, r'$J_{\Theta}(x)$ ist nicht konvex', fontsize=14)
plt.savefig("LogistischeRegression08.png",transparent=True, dpi=300)
plt.show()

Man erkennt direkt, dass lokale Minimima existieren und dass diese Funktion $J$ damit nicht konvex sein kann.
Aus diesem Grund lässt sich das Gradientenverfahren nicht auf die Kostenfunktion anwenden.
Je nachdem, wo man mit der Parameteroptimierung startet, könnte die Suche in ein lokales Minimum laufen und damit die optimalen Modellparameter nicht finden.

Da die Methode der Fehlerquadrate nicht zielführend ist, verwendet man der logistischen Regression eine andere Kostenfunktion.

Die Funktion $\hat{J}_{\Theta}(x)$ verwendet die Logarithmusfunktion angewendet auf $h_{\Theta}(x)$ und in Abhängigkeit von $y$.

$$
\begin{equation*}
\hat{J}_{\Theta}(x)=\begin{cases}
-\log (h_{\Theta}(x)) & \text{falls } y=1\\
-\log (1-h_{\Theta}(x)) & \text{falls } y=0
\end{cases}
\end{equation*}
$$

In [ ]:
xx = np.arange(0.01, 1, .01)

fig, axs =plt.subplots(1, 2, figsize=(9,5))                    


axs[0].plot(xx, -np.log(xx), linewidth=4, label=r'$J_{\Theta}(x)$')
axs[1].plot(xx, -np.log(1-xx), linewidth=4, label=r'$J_{\Theta}(x)$', c='orange')
for i in [0,1]:
    axs[i].spines['right'].set_color('none')
    axs[i].spines['top'].set_color('none')
    axs[i].spines['left'].set_position(('data',0))
    axs[i].spines['bottom'].set_position(('data',0))
    axs[i].xaxis.set_ticks_position('bottom')
    axs[i].yaxis.set_ticks_position('left')
    for label in axs[i].get_xticklabels() + axs[i].get_yticklabels():
        label.set_fontsize(12)
        label.set_bbox(dict(facecolor='white', edgecolor='None', alpha=0.65 ))
    axs[i].legend(loc='upper center', prop={'size': 16})
    axs[i].set_xlabel(r'$h_{\Theta}$', fontsize=16)

axs[0].text(0.2, 2.5, 'Falls y=1', fontsize=18)
axs[1].text(0.2, 2.5, 'Falls y=0', fontsize=18)

plt.savefig("LogistischeRegression09.png",transparent=True, dpi=300)   
plt.show()


Die Kostenfunktion $\hat{J}_{\Theta}(x)$ ist konvex, daher können wir das Gradientenverfahren anwenden.

Um die Fallunterscheidung für $y=0$ und $y=1$ zu eliminieren, können wir $\hat{J}$ auch so formulieren:

$$
\hat{J}_{\Theta}(x)= -y\log(h_{\Theta}(x))- ( (1-y)\log(1-h_{\Theta}(x)))
$$

Um die endgültige Kostenfunktion zu erhalten, skalieren wir die Funktion noch durch die Anzahl der Datenpunkte und erhalten so

$$
J_{\Theta}(x)= \frac{1}{m} -y\log(h_{\Theta}(x))- ( (1-y)\log(1-h_{\Theta}(x)))
$$


Nun können wir das Gradientenverfahren anwenden.
Skalieren wir wie gewohnt zuerst die Werte der Merkmale.

In [ ]:
y = y.reshape(-1,1)

scaling_factors = np.abs(XX[:,1:].max(axis=0)-XX[:,1:].min(axis=0))
scaling_factors = np.concatenate([[1.0], scaling_factors])

X_scaled = XX/scaling_factors
X_scaled[0:5,]

Im folgenden Code-Abschnitt werden die Modell-, Kosten- und Gradient-Descent Funktionen definiert.

In [ ]:
def h(X,theta):
    #print("h -> X:", np.shape(X), "Theta:", np.shape(theta))
    return  1.0 / (1.0+np.exp(-(X@theta)))

def J(X,theta,y):
    yy = h(X,theta)
    return -1/len(y) * (y.T@np.log(yy) + ((1-y).T@np.log(1-yy)))


def gradient_descent(X, y, theta, alpha, iterationen):
    kosten = []
    for iter in range(iterationen):
        costs = J(X, theta, y)
        kosten.append(costs.item(0))
        gradient = 1/len(y) * (X.T @ (sigma(X @ theta) - y))
        theta = theta - (alpha * gradient)
    return theta, kosten

Nun können wir die Parameter des Modells trainieren.

In [ ]:
from sklearn.model_selection import train_test_split


theta0 = np.array([0,0]).reshape(2,1)

X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.3, random_state=0)
X_train/=scaling_factors

y = y.reshape(-1,1)
theta_scaled, kosten = gradient_descent(X_train, y_train, theta0, 8, 1000)
plt.plot(range(1,len(kosten)),kosten[1:], "x-")
plt.xlabel("Epochen", fontsize=14)
plt.savefig("LogistischeRegression10.png",transparent=True, dpi=300)
plt.show()
theta_gd = (theta_scaled.T/scaling_factors).T
theta_gd

Wir testen nun das trainierte Modell mit dem Testdatensatz und bestimmen die Vorhersagegenauigkeit.

In [ ]:
theta_gd
pred_gd = np.array([h(X_test,theta_gd)>=0.5])*1
acc_gd=100-np.sum(np.abs(pred_gd-y_test))*100/len(y_test)
print("Vorhersagegenauigkeit: %.2f%%" % acc_gd)

Nun wollen wir noch testen, zu welchen Ergebnissen die logistische Regression aus der Scikil-Learn Bibliothek kommt.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(XX[:,1:], y.reshape(-1), test_size=0.3, random_state=0)
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X_train, y_train)

logreg.intercept_, logreg.coef_

Wie man sieht, sind die Parameter sehr ähnlich zu denen, die unser "hand-kodiertes" Gradientenverfahren liefert.
Auch die Vorhersagedenauigkeit dieses Modells ist nahezu identisch.

In [ ]:
y_pred = logreg.predict(X_test)
acc_test=100-np.sum(np.abs(y_pred-y_test))*100/len(y_pred)

print("Vorhersagegenauigkeit: %.2f%%" % acc_test)

Nun Plotten wir noch die edgültige Modellfunktion und tragen die Entscheidungsgrenze bei $h_{\Theta}(x)=0.5$ ein.

In [ ]:
theta_gd = (theta_scaled.T/scaling_factors).T

xx = np.linspace(-1,90,300).reshape(-1,1)
xx = np.concatenate(((np.ones(len(xx))).reshape(-1,1), xx), 1)
yy = h(xx,theta_gd)

i = 0
while(yy[i]<0.5): i+=1
xx[:,1][i]

plt.plot(xx[:,1], yy, c='orange', label="Modellfunktion", linewidth=3)
plt.scatter(X, y, label="Datenpunkte")
plt.xlabel("Lernstunden", fontsize=14)
plt.axvline(x=xx[:,1][i], c='r', ls=':', label="Entscheidungsgrenze")
plt.legend(loc='lower right', prop={'size': 12})
plt.savefig("LogistischeRegression11.png",transparent=True, dpi=300)
plt.show()

## Entscheidungsgrenzen

**Aufgabe:** **Verwenden Sie die oben beschrieben Techniken, um Entscheidungsgrenzen für einen zufällig erzeugten Datensatz zu berechnen.**

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt


# generating two-class dataset
X, y = make_blobs(n_samples=100, centers=2, n_features=2, center_box = (-5, 5))


plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

**(a)** Teilen Sie den Datensatz auf (70% Training, 30% Test):
- `X_train`: training dataset
- `X_test`: test dataset
- `y_train`: training labels
- `y_test`: test labels

In [ ]:
X_train, X_test, y_train, y_test = [None]*4
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

assert np.vstack((X_train, X_test)) in X and np.hstack((y_train, y_test)) in y
assert y_train.size/y.size == 0.7

**(b)** Verwenden Sie die *sklearn*-Klasse `LogisticRegression` um ein Modell für den Datensatz zu bilden. Trainieren Sie das Modell mit den oben festgelegten Trainingsdaten.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

assert type(logreg) == LogisticRegression
assert logreg.intercept_, 'Trainieren Sie das Modell mit den Daten!'

Wir visualisieren nun den Datensatz um darzustellen, wie gut unser Modell klassifiziert:

In [ ]:
# Plotting decision regions
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))

Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.Spectral)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)
plt.xlabel(r"$\Theta_0$", fontsize=14)
plt.ylabel(r"$\Theta_1$", fontsize=14)
plt.show()

**(c)** Testen Sie die Vorhersagegenauigkeit (*accuracy*) des Modells mit den Testdaten:

In [ ]:
y_pred = None
acc_test = None

# YOUR CODE HERE
raise NotImplementedError()

print("accuracy: %.2f%%" % acc_test)

In [ ]:
# Test Cell
#----------

assert y_pred.shape == y_test.shape
assert acc_test == 100-np.sum(np.abs(y_pred-y_test))*100/len(y_pred)

In [ ]:
score = logreg.score(X_test, y_test)
print('Test Accuracy Score', score)

Wir wollen nun das Gradientenverfahren verwenden, um unser Modell selbst zu trainieren.

**(d)** Initialisern Sie die Modellparameter $\Theta$:

*Hinweis:* Denken Sie daran, dass der Datensatz 2 Merkmale besitzt. Zusammen mit dem Bias-Parameter sollte $\Theta$ also drei Eintäge haben. Die Dimension von $\Theta$ ist demnach `(3,1)`.

In [ ]:
y_train, y_test = y_train.reshape(-1,1), y_test.reshape(-1, 1)

if X_train[0].size < 3:
    X_train, X_test = np.concatenate((np.ones(y_train.shape) ,X_train), 1),np.concatenate((np.ones(y_test.shape) ,X_test), 1)


theta0 = None
# YOUR CODE HERE
raise NotImplementedError()
theta0

In [ ]:
# Test Cell
#----------

assert theta0.shape == (3,1)

**(e)** Verwenden Sie nun das Gradientenverfahren mit der oben definierten Methode `gradient_decent`:

*Hinweis:* Experimentieren Sie mit verschiedenen Lernraten und Iterationen. Beobachten Sie die Resultate.

In [ ]:
iterations = 40 #change it
alpha = 1 #change it

theta, costs = None, None
# YOUR CODE HERE
raise NotImplementedError()

plt.plot(range(1,len(costs)) ,costs[1:], "x-")
plt.xlabel("Iterations")
plt.show()

In [ ]:
# Test Cell
#----------

assert len(costs) == iterations
assert theta.shape == theta0.shape

In [ ]:
# Plotting decision regions
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))
grid = np.c_[xx.ravel(), yy.ravel()]
grid1 = np.concatenate((np.ones((grid.shape[0], 1)), grid), 1)

Z = h(grid1, theta) >= 0.5
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.Spectral)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)
plt.xlabel("X0", fontsize=14)
plt.ylabel("X1", fontsize=14)
plt.show()

**(f)** Berechnen Sie die *saccuracy* des trainierten Moells und vergleichen Sie die Qualität mit dem vorherigen (*sklearn*) Modell.

*Hinweis:* Berechnen Sie $h$ für die Datenpunkte im Test-Datensatz und weisen Sie allen Werten für $h$ $<0.5$ das Ergebnis $0$ und allen Werten für $h$ $\geq 0.5$ das Ergebnis $1$ zu.

In [ ]:
y_pred = None
acc_test = None

# YOUR CODE HERE
raise NotImplementedError()

print("accuracy: %.2f%%" % acc_test)

In [ ]:
# Test Cell
#----------

assert y_pred.shape == y_test.shape
assert acc_test == 100-np.sum(np.abs(y_pred-y_test))*100/len(y_pred)

### Multiklassen-Klassifikation 

Die logistische Regression liefert uns Ergebnisse für binäre Zielvariable.
Oftmals wollen wir aber mehr als 2 Klassen unterscheiden.

Eine Möglichkeit, um Multiklassen-Klassifikation mit logistischen Regression umzusetzen ist die sogenannte *One-vs-all Klassifikation*.
Dabei werden für `n` Klassen `n` separate, binäre Klassifikationsprobleme definiert, bei denen jeweils nur die betrachtete Klasse den Zielwert `1` zugeteilt bekommt, und für alle anderen Klassen der Zielwert `0` angenommen wird.

Scikit-Learn unterstüzt Multiklassen-Klassifikation in der Klasse `LogisticRegression` über den Parameter `multi_class`.
Setzt man : `multi_class="ovr"` benutzt die führt die Funktion `fit` je eine logistische Regression für jedes Label nach dem *one-vs-all* (oder auch *one-vs-rest*, ovr) Prinzip aus.

In der folgenden Code-Zelle erzeugen wir 3 Punktwolken.
Alle Punkte einer "Wolke" sollen zu einer bestimmten Klasse gehören.

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
from pandas import DataFrame
# generate 2d classification dataset
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=10)
# scatter plot, dots colored by class value
df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
colors = {0:'red', 1:'blue', 2:'green'}
markers = {0:'o', 1:'x', 2:'^'}
fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, marker=markers[key], color=colors[key])
    
plt.legend(loc='upper right', prop={'size': 12})
plt.savefig("LogistischeRegression20.png",transparent=True, dpi=300)
plt.show()

Nun wenden wir ein logistisches Regressionsmodell auf die Datenbasis an.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

logreg = LogisticRegression()


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

pipeline = Pipeline([
    ('scaler',MinMaxScaler()),
    ('model', LogisticRegression(solver='lbfgs',multi_class="ovr"))
])

pipeline.fit(X_train,y_train)

params0 = pipeline.named_steps["model"].intercept_
paramsi  = pipeline.named_steps["model"].coef_

params0, paramsi


Der folgende Graph zeigt die Entscheidungsgrenzen für das Klassifikationsmodell.
Alle Punkte innerhalb eines Bereiches werden der jeweiligen Klasse zugeordnet.

In [ ]:
i=0
xx = np.linspace(X_train[:,0].min()-1, X_train[:,0].max()+1, 300)
yy = np.linspace(X_train[:,1].min()-1, X_train[:,1].max()+1, 300)
XX, YY = np.meshgrid(xx,yy)
ZZ = pipeline.predict(np.c_[XX.ravel(), YY.ravel()])
ZZ = ZZ.reshape(XX.shape)

yyy = params0[0] + paramsi[0][0]*xx + paramsi[0][1]*yy

colors = {0:'red', 1:'blue', 2:'green'}
markers = {0:'o', 1:'x', 2:'^'}
fig, ax = plt.subplots()
grouped = df.groupby('label')
#plt.pcolormesh(XX, YY, ZZ, cmap=plt.cm.Set3)
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, marker=markers[key], color=colors[key])
    

plt.contour(XX, YY, ZZ, cmap=plt.cm.Blues)
plt.legend(loc='upper right', prop={'size': 12})
plt.savefig("LogistischeRegression21.png",transparent=True, dpi=300)
plt.show()